<a href="https://colab.research.google.com/github/MalihaT111/ai-recruitment/blob/label-data-jess/AI_Assisted_Recruitment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/MyDrive/cadence 1a/data"

import os
print(os.listdir(data_path))



['data job posts.csv', 'data job posts.gsheet', 'Resume.csv', 'Resume.gsheet']


In [ ]:
resume_df = pd.read_csv(f"{data_path}/Resume.csv")
resume_df.shape
resume_df.head()
# resume_df.isnull().sum()

ID  \
0  16852973   
1  22323967   
2  33176873   
3  27018550   
4  17812897   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
job_posts_df = pd.read_csv(f"{data_path}/data job posts.csv")
job_posts_df.shape
job_posts_df.head()
# job_posts_df.isnull().sum()

jobpost  \
0  AMERIA Investment Consulting Company\r\nJOB TITLE:  Chief Financial Officer\r\nPOSITION LOCATION: Yerevan, Armenia\r\nJOB DESCRIPTION:   AMERIA Investment Consulting Company is seeking a\r\nChief Financial Officer. This position manages the company's fiscal and\r\nadministrative functions, provides highly responsible and technically\r\ncomplex staff assistance to the Executive Director. The work performed\r\nrequires a high level of technical proficiency in financial management\r\nand investment management, as well as management, supervisory, and\r\nadministrative skills.\r\nJOB RESPONSIBILITIES:  \r\n- Supervises financial management and administrative staff, including\r\nassigning responsibilities, reviewing employees' work processes and\r\nproducts, counseling employees, giving performance evaluations, and\r\nrecommending disciplinary action;\r\n- Serves as member of management team participating in both strategic\r\nand operational planning for the company;\r\n- Directs and oversees the company's financial management activities,\r\nincluding establishing and monitoring internal controls, managing cash\r\nand investments, and managing the investment portfolio in collaboration\r\nwith the Investment team leader. This includes, but is not limited to,\r\nevaluation of investment risk, concentration risk, fund deployment\r\nlevels, adequacy of loss and liquidity reserves Assists investment team\r\nin development of proper documentation and internal systems;\r\n- Directs and oversees the annual budgeting process, including\r\ndeveloping projections for financial planning, and preparing budgets;\r\n- Prepares external and internal financial management reports, such as\r\naudited financial statements, tax returns, and reports for the board of\r\ndirectors and company staff;\r\n- Develops, implements, and maintains efficient and effective accounting\r\nsystems and controls to ensure compliance with national and\r\ninternational accounting standards and principles, sufficiency of fund\r\naccounting, and comprehensiveness of data for reporting and compliance\r\nrequirements;\r\n- Ensures contract compliance, including interpreting and monitoring\r\ncontracts with clients, submitting required reports, and monitoring\r\ncovenants and other contract terms;\r\n- Oversees the design, implementation and maintenance of computer-based\r\ninformation system. Oversees records retention (both manual and\r\ncomputer-based) and file maintenance activities;\r\n- Serves as company's risk manager, including evaluating loss exposure\r\nand obtaining insurance as appropriate;\r\n- Manages other administrative operations, such as facilities\r\nmanagement, payroll administration, office operations, and\r\nadministrative support;\r\n- Monitors corporate compliance with by-laws and articles of\r\nincorporation regarding corporate registration and reporting of\r\nfundraising operations.\r\nREQUIRED QUALIFICATIONS:  To perform this job successfully, an\r\nindividual must be able to perform each essential duty satisfactorily.\r\nThe requirements listed below are representative of the knowledge,\r\nskill, and/or ability required.\r\nKnowledge of:\r\n- Generally accepted accounting principles;\r\n- Local accounting standards and legislation;\r\n- State reporting requirements pertaining to accounting;\r\n- Principles and practices of financial management and budgeting;\r\n- Principles and practices of financial systems design and analysis;\r\n- Principles and practices of contract management, records management,\r\nand risk management;\r\n- Principles and practices of management and supervision;\r\n- Principles and practices of information systems management.\r\nAbility to:\r\n- Apply sound fiscal and administrative practices to the company's\r\nactivities;\r\n- Plan, organize and supervise the work of subordinate employees,\r\nincluding training them, assigning and evaluating their work, and\r\nproviding job performance feedback;\r\n- Critically 

In [ ]:
#Columns with missing values, turning them into a list
condition = job_posts_df.isnull().sum() != 0
job_posts_df.isnull().sum()[condition].index
columnlist = list(job_posts_df.isnull().sum()[condition].index)
columnlist

['Title',
 'Company',
 'AnnouncementCode',
 'Term',
 'Eligibility',
 'Audience',
 'StartDate',
 'Duration',
 'Location',
 'JobDescription',
 'JobRequirment',
 'RequiredQual',
 'Salary',
 'ApplicationP',
 'OpeningDate',
 'Deadline',
 'Notes',
 'AboutC',
 'Attach']

In [ ]:
job_posts_df[columnlist].dtypes
#Everything is object so meaning there's no numerical value

,0
Title,object
Company,object
AnnouncementCode,object
Term,object
Eligibility,object
Audience,object
StartDate,object
Duration,object
Location,object
JobDescription,object


In [ ]:
#for column in columnlist:
    #print(job_posts_df[column].unique())



In [ ]:
#outliers

## Addressing Null values in job_posts_df
Since there are many columns in job_posts_df with null values, we can use reasoning to drop some of the rows or columns. Some columns can be cut if they have too many null values or if they are not really relevant to the problem.  For example, the column "AnnouncementCode" has 17793 null values and the unique non-null values are a random string of letters. It was most likely used to identify the job posting on its original website. As well, "Opening Date" and "Deadline" may not have many null values, but these values might not be very useful to determine whether or not a candidate would be a good fit for a job.

In [ ]:
columns_to_drop = ['AnnouncementCode', 'Term', 'Eligibility', 'Audience', 'StartDate', 'Duration', 'OpeningDate', 'Deadline', 'Notes', 'Attach']
job_posts_df = job_posts_df.drop(columns=columns_to_drop)
job_posts_df.isnull().sum()

,0
jobpost,0
date,0
Title,28
Company,7
Location,32
JobDescription,3892
JobRequirment,2522
RequiredQual,484
Salary,9379
ApplicationP,60


There are still null values, but the columns are too contextually important to the ML problem. We can drop the examples that have null values in these columns since they most likely do not have the information we need to train the model accurately. After we drop these rows, our job_posts_df dataset no longer has any null values.

In [ ]:
columns_to_check = ['Title', 'JobDescription', 'JobRequirment', 'Company', 'Location', 'RequiredQual', 'Salary', 'AboutC']

job_posts_df = job_posts_df.dropna(subset=columns_to_check)

job_posts_df.shape

(5459, 14)

In [ ]:
job_posts_df.isnull().sum()

,0
jobpost,0
date,0
Title,0
Company,0
Location,0
JobDescription,0
JobRequirment,0
RequiredQual,0
Salary,0
ApplicationP,0


In [ ]:
#Removing duplicates
job_posts_df = job_posts_df.drop_duplicates()
print(job_posts_df.duplicated().sum())
resume_df = resume_df.drop_duplicates()
print(resume_df.duplicated().sum())

0
0


In [ ]:
#Check which column has HTML tags
import re
def has_html(text):
    if isinstance(text, str):
        return bool(re.search(r'<.*?>', text))
    return False

In [ ]:
columns_with_html = [col for col in job_posts_df.columns if job_posts_df[col].apply(has_html).any()]
print(columns_with_html)

columns_with_html = [col for col in resume_df.columns if resume_df[col].apply(has_html).any()]
print(columns_with_html)
#So no column in job_post has any HTML tags we need to remove. Only resume :/

[]
['Resume_str', 'Resume_html']


In [ ]:
import re
def clean_html(text):
  return re.sub('<[^<]+?>', '', text)

resume_df['Resume_html'] = resume_df['Resume_str'].apply(clean_html)

In [ ]:
resume_df[['Resume_html', 'Resume_str']].head()
#They're the same thing so i'm dropping the HTML column :D
resume_df.drop(columns=['Resume_html'], inplace= True)

# Checkpoint #2 - Text Normalization
Apply tokenization, lowercasing, stopword removal, and lemmatization.

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
# #Define the vectorizer
# vectorizer = TfidfVectorizer(
#     lowercase=True,
#     stop_words='english',
#     strip_accents='unicode'
# )
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Make sure NLTK resources are available
nltk.download("punkt_tab")

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("stopwords")

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def lemmatize_and_tokenize(text):
    tokens = word_tokenize(str(text).lower())   # lowercase + tokenize
    tokens = [lemmatizer.lemmatize(t) for t in tokens
              if t.isalpha() and t not in stop_words]  # keep only words, no stopwords
    return tokens

# ✅ Define vectorizer with custom tokenizer
vectorizer = TfidfVectorizer(tokenizer=lemmatize_and_tokenize)

# Important: pass only the resume text column
tfidf_matrix_resumes = vectorizer.fit_transform(resume_df["Resume_str"])

print("Shape of tfidf_matrix_resumes:", tfidf_matrix_resumes.shape)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Shape of tfidf_matrix_resumes: (2484, 32389)


In [ ]:
#Fit/transform our the resumes

# This does everything in one call:
# 1. Normalizes (lowercase, stopwords, lemmatization)
# 2. Creates the vocabulary
# 3. Calculates TF-IDF vectors
tfidf_matrix_resumes = vectorizer.fit_transform(resume_df)

print("Shape of tfidf_matrix_resumes:", tfidf_matrix_resumes.shape)

Shape of tfidf_matrix_resumes: (3, 2)


In [ ]:


# Pick a few resumes to inspect
sample_indices = [0, 1, 2]  # change these to any resume indices you want
pd.set_option("display.max_colwidth", None)
for idx in sample_indices:
    original = resume_df["Resume_str"].iloc[idx][:500]  # first 500 chars only for readability
    normalized = lemmatize_and_tokenize(resume_df["Resume_str"].iloc[idx])

    print(f"=== Resume {idx} ===")
    print("Original snippet:")
    print(original, "\n")
    print("Normalized tokens:")
    print(normalized[:30], "...")  # show first 30 tokens
    print("\n" + "="*60 + "\n")


=== Resume 0 ===
Original snippet:
         HR ADMINISTRATOR/MARKETING ASSOCIATE

HR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-task 

Normalized tokens:
['hr', 'associate', 'hr', 'administrator', 'summary', 'dedicated', 'customer', 'service', 'manager', 'year', 'experience', 'hospitality', 'customer', 'service', 'management', 'respected', 'builder', 'leader', 'team', 'strives', 'instill', 'shared', 'enthusiastic', 'commitment', 'customer', 'service', 'highlight', 'focused', 'customer', 'satisfaction'] ...


=== Resume 1 ===
Original snippet:
         HR SPECIALIST, US HR OPERATIONS       S

In [ ]:
# # Show the full text in each cell
# pd.set_option("display.max_colwidth", None)

# # Now when you print, you’ll see everything
# resume_df["Resume_str"].head()



In [ ]:
#next step is to build the model ?



In [ ]:
# See the structure
print(job_posts_df.info())

# See a few sample job posts



<class 'pandas.core.frame.DataFrame'>
Index: 5448 entries, 185 to 18999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jobpost         5448 non-null   object
 1   date            5448 non-null   object
 2   Title           5448 non-null   object
 3   Company         5448 non-null   object
 4   Location        5448 non-null   object
 5   JobDescription  5448 non-null   object
 6   JobRequirment   5448 non-null   object
 7   RequiredQual    5448 non-null   object
 8   Salary          5448 non-null   object
 9   ApplicationP    5448 non-null   object
 10  AboutC          5448 non-null   object
 11  Year            5448 non-null   int64 
 12  Month           5448 non-null   int64 
 13  IT              5448 non-null   bool  
dtypes: bool(1), int64(2), object(11)
memory usage: 601.2+ KB
None
                         Title  \
185  Secretary / office helper   
242           Cheif Accountant   
271       Sales repres

In [ ]:
pd.set_option("display.max_colwidth", None)
job_posts_df.head(5)

jobpost  \
185                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           LadyDalieda\r\nTITLE:  Secretary / office helper\r\nOPEN TO/ ELIGIBILITY CRITERIA:  All capable people\r\nSTART DATE/ TIME:  Immediately\r\nDURATION:  Permanent\r\nLOCATION:  Yerevan, Armenia\r\nJOB DESCRIPTION:  Secretary office helper, Typing transitions from\r\nEnglish to Armenian, able to speck and understand English, Armenian and\r\nRus